| 라이브러리                                                       | 주요 특징                                    | 장점                                                                                                        | 단점                                                                                         | 주요 용도                                                                           |
| :---------------------------------------------------------- | :--------------------------------------- | :-------------------------------------------------------------------------------------------------------- | :----------------------------------------------------------------------------------------- | :------------------------------------------------------------------------------ |
| **TextBlob**                                                | 사전 기반 감성 분석 + 품사 태깅, 명사구 추출 등 NLP 기능 제공  | - 사용이 매우 간단함 (`TextBlob(text).sentiment`)  <br> - 문법 교정, 번역, 품사 분석 등 추가 기능 풍부 <br> - 긍정/부정 + 주관성 점수 함께 제공 | - 감성 어휘 사전이 영어 중심으로 구성됨 <br> - 비격식적 문장(예: SNS 언어)에 대한 정확도 낮음 <br> - 문맥 파악이 약함 (단어 단위로만 판단) | - 기본적인 감정 분석 실습 <br> - 뉴스/리뷰의 전반적 감정 경향 파악 <br> - 학습용 or 프로토타입 단계               |
| **AFINN**                                                   | 단어별 감정 점수(-5~+5)를 사전에 저장해 합산하는 단순한 규칙 기반 | - 속도가 매우 빠름 <br> - 결과가 직관적 (단어 점수의 합으로 계산) <br> - 가벼운 분석에 적합                                              | - 문맥을 고려하지 않음 (부정어, 반어문 처리 불가) <br> - 감정 단어 사전이 제한적임 <br> - 문장 구조나 품사 고려 불가                | - 트윗, 댓글 등 짧은 문장 감성 스코어링 <br> - 실시간 감정 지수 계산(간단한 분석)                            |
| **VADER (Valence Aware Dictionary and sEntiment Reasoner)** | SNS·리뷰 데이터용으로 설계된 감성사전 기반 모델             | - 감탄사, 대문자, 이모티콘 등 비정형 언어 인식 <br> - 부정어(NOT, never 등) 문맥 반영 가능 <br> - 소셜미디어 텍스트에 높은 정확도                   | - 영어 전용 <br> - 긴 문장이나 복잡한 문맥 해석 한계 <br> - 사용자 정의 어휘 추가는 다소 번거로움                            | - 트위터, 유튜브 댓글 등 SNS 텍스트 분석 <br> - 리뷰 감성 분석 (긍/부정 비율 계산) <br> - 실시간 여론 분석 대시보드 등 |


#### 📌 요약 정리
- TextBlob → 감정 + 문법 등 기본 NLP 연습용
- AFINN → 단순한 점수 기반 감성 측정용
- VADER → SNS·리뷰 데이터에 특화된 실제 분석용

In [7]:
import nltk
from nltk.corpus import movie_reviews
from textblob import TextBlob
from afinn import Afinn
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from sklearn.metrics import accuracy_score
import pandas as pd

#nltk 다운로드
nltk.download('movie_reviews', quiet=True)
nltk.download('vader_lexicon', quiet=True)

# 영화 리뷰 데이터 로드
fileids = movie_reviews.fileids()
reviews = [movie_reviews.raw(fileid) for fileid in fileids[:500]] + \
            [movie_reviews.raw(fileid) for fileid in fileids[-500:]] # 앞에서 500개 뒤에서 500개
categories = [movie_reviews.categories(fileid)[0] for fileid in fileids[:500]] +\
                [movie_reviews.categories(fileid)[0] for fileid in fileids[-500:]]  # 앞에서 500개 뒤에서 500개


In [8]:
len(reviews) , categories.count('pos') , categories.count('neg')

(1000, 500, 500)

In [9]:
# 1. TextBlob 사용

def sentiment_testblob(docs):
    return [ 'pos' if TextBlob(doc).sentiment.polarity > 0 else 'neg' for doc in docs]
predictions_texblob = sentiment_testblob(reviews)
accuracy_textblob = accuracy_score(categories,predictions_texblob)
print(f'정확도 : {accuracy_textblob : .1f}')

정확도 :  0.6


In [11]:
# 2. AFINN 사용

def sentiment_afinn(docs):
    afn = Afinn(emoticons=True)
    return [ 'pos' if afn.score(doc) > 0 else 'neg' for doc in docs ]
predictions = sentiment_afinn(reviews)
accuracy = accuracy_score(categories , predictions)
print(f'정확도 : {accuracy : .1f}')

정확도 :  0.7


In [14]:
# 3. vadar

def sentiment_vader(docs):
    analyzer = SentimentIntensityAnalyzer()
    return [ 'pos' if analyzer.polarity_scores(doc)['compound'] > 0 else 'neg' for doc in docs ]
predictions = sentiment_vader(reviews)
accuracy = accuracy_score(categories , predictions)
print(f'정확도 : {accuracy : .1f}')

정확도 :  0.6


## - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -

## 나이브 베이즈

### "좋다" 단어를 본 후 이 리뷰가 긍정일 확률

- 우리가 알고 싶은 것:  
  - **P(긍정 | "좋다")** = "리뷰에 '좋다'가 있을 때, 이 리뷰가 긍정일 확률"

- 베이즈 공식 적용:
  - **P(긍정 | "좋다") = P("좋다" | 긍정) × P(긍정) / P("좋다")**

- 각 항의 의미:
  - **P("좋다" | 긍정)** : 긍정 리뷰들 중에서 "좋다"가 나올 확률  
  - **P(긍정)** : 전체 리뷰 중에서 긍정 리뷰 비율  
  - **P("좋다")** : 전체 리뷰에서 "좋다" 단어가 등장할 확률  


| 구분 | 내용 |
|:-----|:-----|
| **주요 용도** | - 이메일 **스팸 vs 정상** 분류<br> - 영화/상품 리뷰 **긍정 vs 부정 감성 분석**<br> - 뉴스/문서 **카테고리 분류** (정치, 경제, 스포츠 등)<br> - 고차원 텍스트 데이터의 빠른 분류 베이스라인 모델 |
| **장점** | - 계산이 매우 빠르고 구현이 간단함<br> - 적은 데이터로도 어느 정도 성능이 나옴<br> - 고차원(단어 수가 많은) 데이터에 강함<br> - 과적합 위험이 상대적으로 낮고 해석이 직관적 |
| **단점** | - 특성(단어)들이 서로 **독립**이라고 가정 → 현실과 다를 수 있음<br> - 문맥, 반어법, 복합 표현에 약함 (미묘한 감정 표현 해석 한계)<br> - 특정 단어가 한 클래스에만 많이 있을 경우 확률이 극단적으로 치우칠 수 있음 |

#### 📌 요약 정리
- 나이브 베이즈 → “빠르고 실용적인 분류 모델” (머신러닝 접근)

In [ ]:
# 머신러닝 기반 감성분석


from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report



In [17]:
# 데이터 분할
dataset = train_test_split(reviews, categories, test_size=0.2,random_state=42, stratify=categories)
len(dataset[0]),len(dataset[2])

(800, 800)

In [18]:
# tf-idf 벡터화
vectorizer =  TfidfVectorizer(max_features=1000)
x_train = vectorizer.fit_transform(dataset[0])
x_test = vectorizer.transform(dataset[1])

y_train = dataset[2]
y_test = dataset[3]

In [19]:
# 1. Mnb
mnb_clf = MultinomialNB()
mnb_clf.fit(x_train,y_train)
predict = mnb_clf.predict(x_test)
print( classification_report(y_test, predict))

              precision    recall  f1-score   support

         neg       0.77      0.82      0.79       100
         pos       0.81      0.75      0.78       100

    accuracy                           0.79       200
   macro avg       0.79      0.78      0.78       200
weighted avg       0.79      0.79      0.78       200



In [20]:
# 2. LogisticRegression
logi_clf = LogisticRegression()
logi_clf.fit(x_train, y_train)
predict = logi_clf.predict(x_test)
print(classification_report(y_test , predict))

              precision    recall  f1-score   support

         neg       0.79      0.79      0.79       100
         pos       0.79      0.79      0.79       100

    accuracy                           0.79       200
   macro avg       0.79      0.79      0.79       200
weighted avg       0.79      0.79      0.79       200



In [23]:
# 성능향상
# 소문자변환 - 연속된 문자열중에 3글자 이상 - 어간추출(형태소분석) - 불용어 제거
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.tokenize import RegexpTokenizer

def custom_tokenizer(text):
    text = text.lower()
    tokenizer =  RegexpTokenizer(r"[\w']{3,}")
    tokens = tokenizer.tokenize(text)
    porter = PorterStemmer()
    stop_words = set(stopwords.words('english'))
    return [porter.stem(token) for token in tokens if token not in stop_words]
vector = TfidfVectorizer(
    tokenizer  = custom_tokenizer
    ,max_features=1000
    ,min_df=5
    ,max_df=0.5
    ,token_pattern = r"[\w']{3,}"
)
x_train = vector.fit_transform(dataset[0])
x_test = vector.transform(dataset[1])

def evaluate_model(model):    
    model.fit(x_train,y_train)
    predict = model.predict(x_test)
    print( classification_report(y_test, predict))
    
evaluate_model(LogisticRegression())

              precision    recall  f1-score   support

         neg       0.83      0.86      0.85       100
         pos       0.86      0.83      0.84       100

    accuracy                           0.84       200
   macro avg       0.85      0.84      0.84       200
weighted avg       0.85      0.84      0.84       200

